In [4]:
import tensorflow as tf
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import datasets
import numpy as np
from sklearn.feature_selection import chi2
from sklearn.feature_selection import SelectKBest
 
vectorizer = TfidfVectorizer()
labels = list(datasets.fetch_20newsgroups(subset = 'train').target) + list(datasets.fetch_20newsgroups(subset = 'test').target)
data = vectorizer.fit_transform(pickle.load(open('save.p','rb')))


best = SelectKBest(chi2,2000)
data = best.fit_transform(data,labels)

label =np.zeros((data.shape[0],20))
 
for i in range(len(labels)):
    label[i][int(labels[i])] = 1

labels = label
dimensions = data.shape[1]
 
#Number of nodes in each layer
n_nodes_hl1 = 300
n_nodes_hl2 = 400
# n_nodes_hl3 = 400
 
n_classes = 20
batch_size = 100
 
train_size = 15000
 
x = tf.placeholder('float')
y = tf.placeholder('float')
 
 
# creating the neural network
def neural_network_model(data):
    hidden_1_layer = {'weights': tf.Variable(tf.random_normal([dimensions, n_nodes_hl1])),
                      'biases': tf.Variable(tf.random_normal([n_nodes_hl1]))}
 
    hidden_2_layer = {'weights': tf.Variable(tf.random_normal([n_nodes_hl1, n_nodes_hl2])),
                      'biases': tf.Variable(tf.random_normal([n_nodes_hl2]))}

#     hidden_3_layer = {'weights': tf.Variable(tf.random_normal([n_nodes_hl2, n_nodes_hl3])),
#                       'biases': tf.Variable(tf.random_normal([n_nodes_hl3]))}
 
 
    output_layer = {'weights': tf.Variable(tf.random_normal([n_nodes_hl2, n_classes])),
                     'biases': tf.Variable(tf.random_normal([n_classes]))}
 
    l1 = tf.add(tf.matmul(data,hidden_1_layer['weights']), hidden_1_layer['biases'])
    l1 = tf.nn.sigmoid(l1)
 
    l2 = tf.add(tf.matmul(l1,hidden_2_layer['weights']), hidden_2_layer['biases'])
    l2 = tf.nn.sigmoid(l2)

#     l3 = tf.add(tf.matmul(l2,hidden_3_layer['weights']), hidden_3_layer['biases'])
#     l3 = tf.nn.sigmoid(l3)
 
    output = tf.add(tf.matmul(l2, output_layer['weights']), output_layer['biases'])
 
    return output
 
    
def train_neural_network(x):
    prediction = neural_network_model(x)
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = prediction,labels= y))
    optimizer = tf.train.AdamOptimizer().minimize(cost)
 
    hm_epochs = 10
 
    with tf.Session() as sess:
        sess.run(tf.initialize_all_variables())
 
        for epoch in range(hm_epochs):
            epoch_loss = 0
            for _ in range(int(train_size/batch_size)):
                x1 = data[_*batch_size:(_+1)*batch_size].toarray()
                y1 = labels[_*batch_size:(_+1)*batch_size]
                _,c = sess.run([optimizer, cost], feed_dict = {x:x1, y:y1})
                epoch_loss += c
            print ('Epoch : ',epoch,' loss : ',epoch_loss)
 
         
 
        correct = tf.equal(tf.argmax(prediction,1),tf.argmax(y,1))
        accuracy = tf.reduce_mean(tf.cast(correct,'float'))
        
        X1 = data[train_size:18000].toarray()
        print('Accuracy : ',accuracy.eval({x:X1, y:labels[train_size:18000]}))
 
 
train_neural_network(x)

Epoch :  0  loss :  944.1024439334869
Epoch :  1  loss :  297.1887913942337
Epoch :  2  loss :  202.6823816895485
Epoch :  3  loss :  152.6083642244339
Epoch :  4  loss :  123.41187518835068
Epoch :  5  loss :  104.55088210105896
Epoch :  6  loss :  91.37153920531273
Epoch :  7  loss :  81.6273082792759
Epoch :  8  loss :  74.03055825829506
Epoch :  9  loss :  67.88388274610043
Accuracy :  0.81


In [16]:
data[:20]

<20x122277 sparse matrix of type '<class 'numpy.float64'>'
	with 2566 stored elements in Compressed Sparse Row format>

In [19]:

import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
 
 
mnist = input_data.read_data_sets("/tmp/data/", one_hot = True)
 
#Number of nodes in each layer
n_nodes_hl1 = 500
n_nodes_hl2 = 500
n_nodes_hl3 = 500
 
n_classes = 10
batch_size = 500
 
x = tf.placeholder('float',[None, 784])
y = tf.placeholder('float')
 
 
# creating the neural network
def neural_network_model(data):
    hidden_1_layer = {'weights': tf.Variable(tf.random_normal([784, n_nodes_hl1])),
                      'biases': tf.Variable(tf.random_normal([n_nodes_hl1]))}
 
    hidden_2_layer = {'weights': tf.Variable(tf.random_normal([n_nodes_hl1, n_nodes_hl2])),
                      'biases': tf.Variable(tf.random_normal([n_nodes_hl2]))}
 
    hidden_3_layer = {'weights': tf.Variable(tf.random_normal([n_nodes_hl2, n_nodes_hl3])),
                      'biases': tf.Variable(tf.random_normal([n_nodes_hl3]))}
 
 
    output_layer = {'weights': tf.Variable(tf.random_normal([n_nodes_hl3, n_classes])),
                     'biases': tf.Variable(tf.random_normal([n_classes]))}
 
    l1 = tf.add(tf.matmul(data,hidden_1_layer['weights']), hidden_1_layer['biases'])
    l1 = tf.nn.relu(l1)
 
    l2 = tf.add(tf.matmul(l1,hidden_2_layer['weights']), hidden_2_layer['biases'])
    l2 = tf.nn.relu(l2)
 
    l3 = tf.add(tf.matmul(l2,hidden_3_layer['weights']), hidden_3_layer['biases'])
    l3 = tf.nn.relu(l3)
 
    output = tf.add(tf.matmul(l3, output_layer['weights']), output_layer['biases'])
 
    return output
 

def train_neural_network(x):
    prediction = neural_network_model(x)
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = prediction,labels= y))
    optimizer = tf.train.AdamOptimizer().minimize(cost)
 
    hm_epochs = 10
 
    with tf.Session() as sess:
        sess.run(tf.initialize_all_variables())
 
        for epoch in range(hm_epochs):
            epoch_loss = 0
            for _ in range(int(mnist.train.num_examples/batch_size)):
                x1,y1 = mnist.train.next_batch(batch_size)
                _,c = sess.run([optimizer, cost], feed_dict = {x:x1, y:y1})
                epoch_loss += c
            print ('Epoch : ',epoch,' loss : ',epoch_loss)
 
         
 
        correct = tf.equal(tf.argmax(prediction,1),tf.argmax(y,1))
        accuracy = tf.reduce_mean(tf.cast(correct,'float'))
 
        print('Accuracy : ',accuracy.eval({x:mnist.test.images, y:mnist.test.labels}))
 
 
train_neural_network(x)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Epoch :  0  loss :  618050.3696289062
Epoch :  1  loss :  162300.60131835938
Epoch :  2  loss :  103822.30633544922
Epoch :  3  loss :  73081.59210205078
Epoch :  4  loss :  53386.48406982422
Epoch :  5  loss :  40074.320388793945
Epoch :  6  loss :  30518.789237976074
Epoch :  7  loss :  22886.56457901001
Epoch :  8  loss :  17320.856742858887
Epoch :  9  loss :  12990.381008148193
Accuracy :  0.9273


In [15]:
mnist.test.images[:10]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)